![Curso Schwarz-Sosa-Suriano](http://www.fi.uba.ar/sites/default/files/logo.png)
# Integración Numérica - Segunda parte
**Curso Schwarz - Sosa - Suriano**
- Métodos Numéricos. *Curso 2*
- Análisis Numérico I. *Curso 4*
- Métodos Matemáticos y Numéricos. *Curso 6*

## Fórmulas Compuestas

$$ \int_a^b f(x) \approx Q_n = \sum_0^n c_i . f(x_i)$$

Puesto que el orden de convergencia de los métodos que hemos visto - $O(h^{orden})$- se mide en términos de potencias del paso $h = \frac{b-a}{n}$, a medida que vamos eligiendo un método con más puntos, tenemos dos efectos a nuestro favor.

En primer lugar, al ser fijo el intervalo $[a,b]$ al aumentar n se reduce el valor de h. Y al mismo tiempo, al aumentar el grado del polinomio interpolante, aumenta el orden de convergencia del método.

Ahora bien, para que el término $O(h^{orden})$ sea cada vez más se necesita que $h$ sea lo suficientemente pequeño, y en este sentido no basta con tomar más puntos de integración, ya que generaríamos fórmulas que dependerían de polinomios de grado alto.

Es por eso que a la hora de intentar obtener mejores aproximaciónes se utilizan las denominadas Fórmulas Compuestas, que dividen el intervalo $[a,b]$ en intervalos más pequeños y luego aplican reiteradas veces las fórmulas vistas en cada uno de esos intervalos. 

In [ ]:
import numpy as np
import pandas as pd
import sympy as sm
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output


def f(x):
    return x*np.sin(x)+x

a, b = 1,10

x = sm.symbols('x')
primitiva = sm.integrate(x * sm.sin(x) + x,x)
primitivaN = sm.lambdify(x,primitiva)
integralExacta = primitivaN(b)-primitivaN(a)
print ('Integral exacta ',integralExacta)

def ErrorRel(aprox):
    e=100*abs((aprox-integralExacta)/aprox)
    return e

In [ ]:
def graficoActualizar(a,b,f,q,k):
    ax.clear()
    
    gap = 0.2 * (b-a)
    
    fig.set_figheight(10)
    fig.set_figwidth(15)
    
    x = np.linspace(a-gap, b+gap)
    y = f(x)
    ax.plot(x, y, 'r', linewidth=1)  
    
    ax.set_ylim(bottom=0)

    fig.text(0.9, 0.1, '$x$')
    fig.text(0.12, 0.9, '$y$')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')

    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    ax.set_yticks([])

    h = (b-a) / k
    
    for i in range(0, k):
        x0 = a + i*h
        x1 = x0 + h
        
         # Cuadratura
        xq = np.linspace(x0, x1)
        iq = q(xq,x0,x1)
        ax.plot(xq, iq, 'k', linewidth=1)

        # Función
        iy = f(xq)
        ax.fill_between(xq, 0, iq, facecolor='#0099cc', edgecolor ='k')
        ax.fill_between(xq, iq, iy, hatch = '//', alpha='0.1')    

### Fórmulas Compuestas de Newton-Cotes

Aplicar en forma compuesta las fórmulas abiertas no reviste ninguna particularidad especial, ya que los puntos interiores de un intervalo nunca coinciden con los del intervalo siguiente. Es decir que la resolución solamente exige repetir para cada intervalo el método visto, por ejemplo, en el Punto Medio:

In [ ]:
def PuntoMedioC(a,b,k):
    h = (b-a) / k  
    Q = 0
    for i in range (0,k):
        Q += h * f(a+i*h+h/2) 
    return Q
    
def q(x,a,b):
    h = b-a
    return f(a+h/2)*x/x

slider = widgets.IntSlider(value=1, min=1, max=20, step=1, description=('Intervalos'));

def on_slider_change(change):
    clear_output(wait=True)
    k = change['new']
    graficoActualizar(a,b,f,q,k)
    display (fig)
    display (slider)
    print ('Aproximación', PuntoMedioC(a,b,k))
    print ('Error', ErrorRel(PuntoMedioC(a,b,k)))
    print ('Paso', (b-a)/k)    
slider.observe(on_slider_change, names='value')
fig, ax = plt.subplots()
plt.close(fig)  

In [ ]:
on_slider_change({'new':1})

Desde el mismo punto de vista, por ejemplo, el método de los rectángulos compuesto, tampoco presenta ninguna particularidad como fórmula compuesta, más allá de volverse un proceso reiterativo respecto del visto para el Rectángulo.

In [ ]:
def RectAC(a,b,k): #lo hacemos solamente para la versión en exceso
    h = (b-a) / k  
    Q = 0
    for i in range (0,k):
        Q += h * f(a+i*h) 
    return Q
    
def q(x,a,b):
    h = b-a
    return f(a)*x/x

slider = widgets.IntSlider(value=1, min=1, max=20, step=1, description=('Intervalos'));

def on_slider_change(change):
    clear_output(wait=True)
    k = change['new']
    graficoActualizar(a,b,f,q,k)
    display (fig)
    display (slider)
    print ('Aproximación', RectAC(a,b,k))
    print ('Error', ErrorRel(RectAC(a,b,k)))
    print ('Paso', (b-a)/k)
    
slider.observe(on_slider_change, names='value')

fig, ax = plt.subplots()
plt.close(fig)  

In [ ]:
on_slider_change({'new':1})

En el caso de las fórmulas cerradas de 2 o más puntos, en cambio, siempre tendremos la particularidad que el extremo derecho de un intervalo coincide con el izquierdo del siguiente. Eso lleva a que en la bibliografía aparezcan para los métodos de Trapecio y Simpson, expresiones compuestas que no implican solamente repetir la fórmula simple de cada uno de esos métodos.

### Método del Trapecio Compuesto

En el caso del Trapecio compuesto se divide al intervalo $[a,b]$ en $k$ intervalos más pequeños. Al aplicar la fórmula simple en cada intervalo se descubre que para cualquier extremo de esos intervalos menores (salvo para $a$ y $b$), se suma 2 veces el término $c_i.f(x_i)$. De modo que la expresión queda:

$$TC(k) = \frac{1}{2} . h . f(x_0) + \sum_{i=1}^{k-1} h . f(x_i) + \frac{1}{2} . h . f(x_k)$$ 

donde

$$ h = \frac{b-a}{k} \text{, }  x_0 = a \text{, } x_k = b$$

In [ ]:
def TrapecioC(a,b,k):
    h = (b-a) / k
    Q = (h/2) * f(a)     
    for i in range (1,k):
        Q += h * f(a+i*h) 
    Q += (h/2) * f(b)
    return Q
    
def q(x,a,b):
    h = b-a
    F01 = (f(b) - f(a)) / h 
    return f(a) + F01 * (x-a)

slider = widgets.IntSlider(value=1, min=1, max=20, step=1, description=('Intervalos'));

def on_slider_change(change):
    clear_output(wait=True)
    k = change['new']
    graficoActualizar(a,b,f,q,k)
    display (fig)
    display (slider)
    print ('Aproximación', TrapecioC(a,b,k))
    print ('Error', ErrorRel(TrapecioC(a,b,k)))
    print ('Paso', (b-a)/k)
    
slider.observe(on_slider_change, names='value')

fig, ax = plt.subplots()
plt.close(fig)  

In [ ]:
on_slider_change({'new':1})

### Método de Simpson Compuesto

En este caso, vamos a considerar la cantidad de intervalos $k$ como la cantidad de espacios en los que aplicaríamos Simpson, por lo que $h=\frac{b-a}{2.k}$

En algunas bibliografías se adopta directamente una cantidad de intervalos $j$ par, siendo $h=\frac{b-a}{j}$, por lo que pueden encontrar que el algoritmo difiere levemente del que proponemos aquí:

$$SC(k) = \frac{1}{3} . h . f(x_0) + \sum_{i=1 (impares)}^{2.k-1} \frac{4}{3} .h . f(x_i) + \sum_{i=1 (pares)}^{2.k-1}\frac{2}{3} . h . f(x_i)  + \frac{1}{3} . h . f(x_{2k})$$ 

donde

$$ h = \frac{b-a}{2.k} \text{, }  x_0 = a \text{, } x_{2k} = b$$

In [ ]:
def SimpsonC(a,b,k):
    h = (b-a) / (2*k)
    Q = (1/3) * h * f(a)     
    for i in range (1,2*k):
        if (i % 2 == 0): #par
            Q += (2/3) * h * f(a+i*h) 
        else: #impar
            Q += (4/3) * h * f(a+i*h)             
    Q += (1/3) * h * f(b)
    return Q
    
def q(x,a,b):
    h = (b-a)/2
    F01 = (f(a+h) - f(a)) / h 
    F12 = (f(b) - f(a+h)) / h
    F012 = (F12 - F01) / (2*h)
    return f(a) + F01 * (x-a) + F012 * (x-a) * (x-(a+h))

slider = widgets.IntSlider(value=1, min=1, max=64, step=1, description=('Intervalos'));

def on_slider_change(change):
    clear_output(wait=True)
    k = change['new']
    graficoActualizar(a,b,f,q,k)
    display (fig)
    display (slider)
    print ('Aproximación', SimpsonC(a,b,k))
    print ('Error %', ErrorRel(SimpsonC(a,b,k)))
    print ('Paso', (b-a)/(2*k))
    
slider.observe(on_slider_change, names='value')

fig, ax = plt.subplots()
plt.close(fig)  

In [ ]:
on_slider_change({'new':1})

## Extrapolación de Richardson

Análogamente a lo que vió para Diferenciación Numérica, aquí también la Extrapolación de Richardson permite aprovechar la reducción del paso $h$ en forma sistemática no solo para reducir $O(h^{orden})$ sino también para reducir el $orden$ de nuestra aproximación.

Esta estrategia resulta particularmente popular combinada con el método de los Trapecios, cuyo error de truncamiento es de orden $O(h^2)$, se denomina Método de Romberg.

### Método de Romberg:

$$ R_{1,1} = \frac{b-a}{2} . [f(a)+f(b)] $$

$$ R_{k,1} = \frac{1}{2} \left \{ R_{k-1,1} + h_{k-1} . \sum_{i=1}^{2^{k-2}} f[a+(2.i-1).h_k])  \right \} \hspace{10mm} \text{ con } h_k = \frac{b-a}{2^{k-1}}$$

$$ R_{k,j} = R_{k,j-1} + \frac{R_{k,j-1}-R_{k-1,j-1}}{4^{j-1}-1}$$

In [ ]:
R = np.zeros((1,1))
R[0,0] = ((b-a)/2)*(f(a)+f(b))
print (R)
def Romberg(R):
    k = R.shape[0]
    R = np.vstack((R,np.zeros((1,k))))
    R = np.hstack((R,np.zeros((k+1,1)))) 
    hk = (b-a) / (2**k)
    for i in range (0,2**(k-1)):
        R[k,0] += hk * f(a+(2*i+1)*hk)
    R[k,0] += R[k-1,0] / 2    
    for j in range (1,k+1):
        R[k,j] = R[k,j-1] + (R[k,j-1]-R[k-1,j-1])/(4**j-1)
    return R

In [ ]:
R = Romberg(R)
k = R.shape[0]-1
print (R)
print ('Aproximación',R[k,k])
print ('Error %', ErrorRel(R[k,k]))

In [ ]:
R = Romberg(R)
k = R.shape[0]-1
print (R)
print ('Aproximación',R[k,k])
print ('Error %', ErrorRel(R[k,k]))

In [ ]:
R = Romberg(R)
k = R.shape[0]-1
print (R)
print ('Aproximación',R[k,k])
print ('Error %', ErrorRel(R[k,k]))

In [ ]:
R = Romberg(R)
k = R.shape[0]-1
print (R)
print ('Aproximación',R[k,k])
print ('Error %', ErrorRel(R[k,k]))

In [ ]:
R = Romberg(R)
print(R)
k = R.shape[0]-1
print ('Aproximación',R[k,k])
print ('Error %', ErrorRel(R[k,k]))

In [ ]:
R = Romberg(R)
print(R)
k = R.shape[0]-1
print ('Aproximación',R[k,k])
print ('Error %', ErrorRel(R[k,k]))